#COMO DETECTAR DISTACIAMENTO SOCIAL UTILIZANDO A BIBLIOTECA OPENCV

Pra começar, vamos se conectar ao ao google drive para pegar nossos arquivos.

In [2]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

/
Mounted at /content/gdrive


In [ ]:
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
!ls /mydrive/VC/distanc_social/*

In [5]:
!cp -r /mydrive/VC/distanc_social/* ../

# Agora a brincadeira vai começar!!!

In [6]:
# importe os pacotes necessários
from lucasdistac import distancia_social_config as config
from lucasdistac.deteccao import detect_pessoa
from scipy.spatial import distance as dist
import numpy as np
import argparse
import imutils
import cv2
import os

In [7]:
# carregue os rótulos da classe COCO em que nosso modelo YOLO foi treinado
labelsPath = os.path.sep.join([config.MODEL_PATH, "coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")

# vamos mostrar o caminho do no peso YOLO e o da configuração do modelo
weightsPath = os.path.sep.join([config.MODEL_PATH, "yolov3.weights"])
configPath = os.path.sep.join([config.MODEL_PATH, "yolov3.cfg"])

# Vamos carregar nosso detector de objetos YOLO treinado no conjunto de dados COCO com (80 classes)
print("[INFO] Carregando YOLO...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

[INFO] Carregando YOLO...


In [8]:
# verifique se vamos usar GPU
if config.USE_GPU:
	# definir CUDA como backend e destino preferíveis
	print("[INFO] configurando back-end e destino preferíveis para CUDA...")
	net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
	net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

In [9]:
# determinar apenas os nomes de camada de *saída* que precisamos do YOLO
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [10]:
# inicializar o fluxo de vídeo e o caminho para o arquivo de saída de vídeo
print("[INFO] acessando arquivo de vídeo...")
vs = cv2.VideoCapture("stm.mp4" if "stm.mp4" else 0)
writer = None

[INFO] acessando arquivo de vídeo...


In [11]:
# vamos fazer um loop em cada frame do vídeo
while True:
	# leia o próximo quadro do arquivo
	(grabbed, frame) = vs.read()

	# se o quadro não foi capturado, chegamos ao fim
	# do fluxo
	if not grabbed:
		break

	# redimensione a moldura e, em seguida, detecte pessoas ( somente pessoas e nada mais!)
	frame = imutils.resize(frame, width=700)
	results = detect_pessoa(frame, net, ln,
		personIdx=LABELS.index("person"))

	# inicializar o conjunto de índices que violam a distância social
	violate = set()

	# garantir que haja *pelo menos* detecções de duas pessoas (vamos organizar na
	# ordem para calcular nossos mapas de distância aos pares)
	if len(results) >= 2:
		# extrair todos os centróides dos resultados e calcular as
		# Distâncias euclidianas entre todos os pares de centróides
		centroids = np.array([r[2] for r in results])
		D = dist.cdist(centroids, centroids, metric="euclidean")

		# loop sobre o triangular superior da matriz de distância
		for i in range(0, D.shape[0]):
			for j in range(i + 1, D.shape[1]):
				# verifique se a distância entre quaisquer dois
				# pares de centroides é menor que o número configurado
				# de pixels
				if D[i, j] < config.MIN_DISTANCE:
					# atualizar nosso conjunto de violações com os índices de
					# os pares de centroide
					violate.add(i)
					violate.add(j)

	# repetir os resultados
	for (i, (prob, bbox, centroid)) in enumerate(results):
		# extraia a caixa delimitadora e as coordenadas do centroide, então
		# inicializar a cor da anotação
		(startX, startY, endX, endY) = bbox
		(cX, cY) = centroid
		color = (0, 255, 0)

		# se o par de índices existir dentro do conjunto de violação, então
		# atualize a cor
		if i in violate:
			color = (0, 0, 255)

		# desenhe (1) uma caixa delimitadora ao redor da pessoa e (2) o
		# coordenadas de centroide da pessoa,
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
		cv2.circle(frame, (cX, cY), 5, color, 1)

	# Agora vamos desenhar o número total de violações de distanciamento social no
	# quadro de saída
	text = "Violacao de Distanciamento: {}".format(len(violate))
	cv2.putText(frame, text, (10, frame.shape[0] - 25),
		cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 3)

	# verifique se o quadro de saída deve ser exibido para o nosso
	# tela
	'''if 1 > 0:
		# mostre o frame como saida
		cv2.imshow(frame)
		key = cv2.waitKey(1) & 0xFF

		# pressione a tecla `q` para inerromper o loop
		if key == ord("q"):
			break'''

	# se o caminho do arquivo de vídeo foi fornecido e o vídeo
	# não foi inicializado, faça isso agora
	if "saida.avi" != "" and writer is None:
		# inicializar nosso gravador de vídeo
		fourcc = cv2.VideoWriter_fourcc(*"MJPG")
		writer = cv2.VideoWriter("saida.avi", fourcc, 25,
			(frame.shape[1], frame.shape[0]), True)

	# se o gravador de vídeo não for Nenhum, grave o quadro na saída

	# arquivo de vídeo
	if writer is not None:
		writer.write(frame)

KeyboardInterrupt: ignored

CTRL + SHIFT + I
Para aumentar o tempo de conexão com o colab

```
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
```



# Copie para o seu drive para baixar o vídeo de saida

In [ ]:
!cp ../saida.avi /mydrive/VC/distanc_social/